In [1]:
import os
import torch
import numpy as np
from minimodel import data

device = torch.device('cuda')

In [2]:
mouse_id = 5

data_path = './data'
weight_path = './checkpoints'
np.random.seed(1)

In [3]:
# load images
img = data.load_images(data_path, mouse_id, file=data.img_file_name[mouse_id])

raw image shape:  (68000, 66, 264)
cropped image shape:  (68000, 66, 130)
img:  (68000, 66, 130) -2.0829253 2.1060908 float32


In [4]:
# load neurons
fname = '%s_nat60k_%s.npz'%(data.db[mouse_id]['mname'], data.db[mouse_id]['datexp'])
spks, istim_train, istim_test, xpos, ypos, spks_rep_all = data.load_neurons(file_path = os.path.join(data_path, fname), mouse_id = mouse_id)
n_stim, n_neurons = spks.shape


loading activities from ./data/FX20_nat60k_2023_09_29.npz


In [5]:
# split train and validation set
itrain, ival = data.split_train_val(istim_train, train_frac=0.9)


splitting training and validation set...
itrain:  (43081,)
ival:  (4787,)


In [6]:
# normalize data
spks, spks_rep_all = data.normalize_spks(spks, spks_rep_all, itrain)


normalizing neural data...
finished


In [7]:
ineur = np.arange(0, n_neurons) #np.arange(0, n_neurons, 5)
spks_train = torch.from_numpy(spks[itrain][:,ineur]).to(device)
spks_val = torch.from_numpy(spks[ival][:,ineur]).to(device)

print('spks_train: ', spks_train.shape, spks_train.min(), spks_train.max())
print('spks_val: ', spks_val.shape, spks_val.min(), spks_val.max())

img_train = torch.from_numpy(img[istim_train][itrain]).to(device).unsqueeze(1) # change :130 to 25:100 
img_val = torch.from_numpy(img[istim_train][ival]).to(device).unsqueeze(1)
img_test = torch.from_numpy(img[istim_test]).to(device).unsqueeze(1)

print('img_train: ', img_train.shape, img_train.min(), img_train.max())
print('img_val: ', img_val.shape, img_val.min(), img_val.max())
print('img_test: ', img_test.shape, img_test.min(), img_test.max())

input_Ly, input_Lx = img_train.shape[-2:]

spks_train:  torch.Size([43081, 2746]) tensor(-1.4092e-15, device='cuda:0') tensor(48.7427, device='cuda:0')
spks_val:  torch.Size([4787, 2746]) tensor(-6.8745e-16, device='cuda:0') tensor(44.7361, device='cuda:0')
img_train:  torch.Size([43081, 1, 66, 130]) tensor(-2.0829, device='cuda:0') tensor(2.1061, device='cuda:0')
img_val:  torch.Size([4787, 1, 66, 130]) tensor(-2.0829, device='cuda:0') tensor(2.1061, device='cuda:0')
img_test:  torch.Size([500, 1, 66, 130]) tensor(-2.0829, device='cuda:0') tensor(2.1061, device='cuda:0')


In [8]:
# build model
from minimodel import model_builder
nlayers = 2
nconv1 = 16
nconv2 = 320
model, in_channels = model_builder.build_model(NN=len(ineur), n_layers=nlayers, n_conv=nconv1, n_conv_mid=nconv2)
model_name = model_builder.create_model_name(data.mouse_names[mouse_id], data.exp_date[mouse_id], n_layers=nlayers, in_channels=in_channels)

model_path = os.path.join(weight_path, model_name)
model = model.to(device)

core shape:  torch.Size([1, 320, 33, 65])
input shape of readout:  (320, 33, 65)
model name:  FX20_092923_2layer_16_320_clamp_norm_depthsep_pool_xrange_176.pt


In [9]:
# train model
from minimodel import model_trainer
print(device)

if not os.path.exists(model_path):
    best_state_dict = model_trainer.train(model, spks_train, spks_val, img_train, img_val, device=device)
    torch.save(best_state_dict, model_path)
    print('saved model', model_path)
model.load_state_dict(torch.load(model_path))
print('loaded model', model_path)

cuda
Learning rate = 0.001
epoch 0, train_loss = 0.8419, val_loss = 0.8176, varexp_val = 0.0475, time 53.73s
epoch 1, train_loss = 0.8119, val_loss = 0.8054, varexp_val = 0.0669, time 101.58s
epoch 2, train_loss = 0.8030, val_loss = 0.7968, varexp_val = 0.0794, time 149.44s
epoch 3, train_loss = 0.7978, val_loss = 0.7931, varexp_val = 0.0858, time 197.29s
epoch 4, train_loss = 0.7949, val_loss = 0.7907, varexp_val = 0.0895, time 245.13s
epoch 5, train_loss = 0.7926, val_loss = 0.7903, varexp_val = 0.0906, time 292.92s
epoch 6, train_loss = 0.7908, val_loss = 0.7879, varexp_val = 0.0941, time 340.57s
epoch 7, train_loss = 0.7896, val_loss = 0.7872, varexp_val = 0.0955, time 388.29s
epoch 8, train_loss = 0.7886, val_loss = 0.7865, varexp_val = 0.0962, time 436.01s
epoch 9, train_loss = 0.7876, val_loss = 0.7855, varexp_val = 0.0984, time 483.72s
epoch 10, train_loss = 0.7869, val_loss = 0.7847, varexp_val = 0.0993, time 531.43s
epoch 11, train_loss = 0.7862, val_loss = 0.7848, varexp_val

KeyboardInterrupt: 

In [ ]:
# test model
test_pred = model_trainer.test_epoch(model, img_test)
print('test_pred: ', test_pred.shape, test_pred.min(), test_pred.max())

test_pred:  (500, 2746) 0.0017536283 8.742443


In [ ]:
from minimodel import metrics
test_fev, test_feve = metrics.feve(spks_rep_all, test_pred)

threshold = 0.15
print(f'filtering neurons with FEV > {threshold}')
valid_idxes = np.where(test_fev > threshold)[0]
print(f'valid neurons: {len(valid_idxes)} / {len(test_fev)}')
print(f'FEVE (test): {np.mean(test_feve[test_fev > threshold])}')

filtering neurons with FEV > 0.15
valid neurons: 1239 / 2746
FEVE (test): 0.7267250418663025
